 # <center> Pandas

In [ ]:
import numpy as np
import pandas as pd

pd.set_option("display.precision", 3)

# будем отображать графики прямо в jupyter'e
%matplotlib inline
import matplotlib.pyplot as plt
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 

#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 8, 5

import seaborn as sns

import os

In [ ]:
print("Pandas version:", pd.__version__)
print("Current working directory:", os.getcwd())

In [ ]:
!echo $VIRTUAL_ENV

### План

1. DataFrame, Series, describe (include), shape
2. Создание DataFrame, dtypes
3. Колонка по точке или по скобкам
4. loc, iloc. loc - по индексам, iloc - геометрический проход
5. value_counts
6. Сортировка (sort_index, sort_values)
7. Фильтрация (булева индексация), [и, или]
8. Статистики
9. reindex, обработка пропущенных значений (how)
10. Доступ к функциям типов (str, dt)
11. pandas options
12. groupby, разобрать по строчкам
13. Таблицы сопряженности crosstab (pd.crosstab(data['female'], data['looks']), сводные таблицы pivot_table
14. concat, join, merge, append
15. apply, lambda, map
16. date_range
17. stack, unstack
18. astype('category')
19. read_csv, to_csv, read_excel, to_excel
20. sqlalchemy, read_sql_table, to_sql_table
21. df.info(memory_usage='deep')

Pandas - excel в коде

## Загрузка и первичная оценка данных

SQL Databases, CSV, Excel..

In [ ]:
df = pd.read_csv('./data/telecom_churn.csv')
df.head()

Смотрим число строк (клиентов, примеров) и столбцов (признаков):

In [ ]:
df.shape

Посмотрим названия столбцов

In [ ]:
df.columns

Посмотрим общую информацию по датафрейму и всем признакам. Использование памяти:

In [ ]:
df.info()
#df.info(memory_usage='deep')

В колонке `Dtype` видим типы признаков. Ещё видим, что отсутствуют пропуски в данных во всех строках (наблюдениях) в датасете

In [ ]:
df.memory_usage(deep=True)

In [ ]:
df['Voice mail plan'].nbytes

### describe и простая статистика

По-умолчанию выводится информация только о числовых типах данных

In [ ]:
df.describe()

In [ ]:
df.describe(include=['object', 'bool'])

### Манипуляции столбцами

Обращение/получение столбца через точку работает когда в названии столбца нет спецсимволов

In [ ]:
df.Churn

Обращение через скобки работает всегда

In [ ]:
df['Churn']

Пример преобразования типа столбца для целевого класса:

In [ ]:
df['Churn'] = df['Churn'].astype('int8')

Проверяем распределение целевого класса:

In [ ]:
df['Churn'].value_counts()

In [ ]:
df['Churn'].value_counts(normalize=True)

In [ ]:
df['Churn'].value_counts().plot(kind='bar', label='Churn')
plt.legend()
plt.title('Распределение оттока клиентов');

### Наглядная взаимосвязь признаков

In [ ]:
sns.pairplot(df[['Total day charge','Customer service calls','Churn']], hue='Churn');

In [ ]:
# TODO: выводы?

## Типы данных

In [ ]:
type(df['Voice mail plan'])

In [ ]:
df['Voice mail plan'].dtype

In [ ]:
df['Voice mail plan'].name, df['Voice mail plan'].dtype == np.object


In [ ]:
df['Total day calls'].dtype, df['Total day calls'].dtype == np.int64

In [ ]:
df['Churn'].dtype, np.bool

### `select_dtypes` - отбор колонок на основе типа данных

In [ ]:
df.select_dtypes('float64').head()

Посмотрим на корреляции количественных признаков с помощью метода `corr` и визуализации матрицы корреляций.

In [ ]:
corr_matrix = df.select_dtypes(['float64', 'int64']).corr()

С помощью pyplot можно визуализировать матрицу корреляций. Но для того, чтобы вывести эту же матрицу с полезными аннотациями, надо заморочиться.

In [ ]:
plt.imshow(corr_matrix);

Тут полезной оказывается библиотека `seaborn` - высокоуровневая обёртка над pyplot. Та же самая матрица корреляций выводится уже более наглядно:

In [ ]:
sns.heatmap(corr_matrix);

In [ ]:
# TODO: выводы

## Сортировка

DataFrame можно отсортировать по значению какого-нибудь из признаков.

In [ ]:
df.sort_values(by='Total day charge', 
        ascending=False).head()

Сортировать можно по группе признаков

In [ ]:
df.sort_values(by=['Churn', 'Total day charge'],
        ascending=[True, False]).head()

## Извлечение данных

**1. Какова доля нелояльных клиентов в датасете?**

In [ ]:
df['Churn'].mean()

**2. Сколько в среднем в течение дня разговаривают по телефону нелояльные пользователи?**

In [ ]:
df[df['Churn'] == 1]['Total day minutes'].mean()

In [ ]:
df[df['Churn'] == 0].mean()

**3. Какова максимальная длина международных звонков среди лояльных пользователей (Churn == 0), не пользующихся услугой международного роуминга ('International plan' == 'No')?**

In [ ]:
df[(df['Churn'] == 0) & (df['International plan'] == 'No')]['Total intl minutes'].max()

**4. Какова доля оттока у пользователей с подключенным роумингом и количеством звонков в техподдержку больше 3**

In [ ]:
#TODO: написать запрос

## Индексация и срезы

Датафреймы можно индексировать как по названию столбца или строки, так и по порядковому номеру. Для индексации по названию используется метод `loc`, по номеру — `iloc`.

`loc` - передай нам значения для id строк от 0 до 5 и для столбцов от State до Area code

In [ ]:
df.loc[0:5, 'State':'Area code']

`iloc` - передай нам значения первых пяти строк в первых трёх столбцах

In [ ]:
df.iloc[0:5, 0:3]

Если нам нужна первая или последняя строчка датафрейма, пользуемся конструкцией df[:1] или df[-1:]

In [ ]:
df[:1]

In [ ]:
df[-1:]

### Фильтрация по маске

In [ ]:
mask = (df['Churn'] == 0) & (df['International plan'] == 'Yes')

In [ ]:
df.shape

In [ ]:
df[mask].shape

## Применение функций к ячейкам, столбцам и строкам

### Применение функции к каждому столбцу: apply

In [ ]:
df.apply(np.max)

### Применение функции к каждой строке: apply

Метод apply можно использовать и для того, чтобы применить функцию к каждой строке. Для этого нужно указать axis=1

In [ ]:
rectangles = [
    { 'height': 40, 'width': 10 },
    { 'height': 20, 'width': 9 },
    { 'height': 3.4, 'width': 4 }
]

rectangles_df = pd.DataFrame(rectangles)
rectangles_df
def calculate_area(row):
    return row['height'] * row['width']

rectangles_df['area'] = rectangles_df.apply(calculate_area, axis=1)
rectangles_df

### Замена значений в колонке: map

In [ ]:
d = {'No' : False, 'Yes' : True}
df['International plan'] = df['International plan'].map(d)
df.head()

### Замена значений в колонке: replace

In [ ]:
df = df.replace({'Voice mail plan': d})
df.head()

In [ ]:
df['International plan'].dtype, df['Voice mail plan'].dtype

### Строковые функции

In [ ]:
df['State len'] = df['State'].str.len()

In [ ]:
df.head()

# Группировка данных

In [ ]:
columns_to_show = ['Total day minutes', 'Total eve minutes', 'Total night minutes']
df.groupby(['Churn'])[columns_to_show].describe(percentiles=[]).T

In [ ]:
columns_to_show = ['Total day minutes', 'Total eve minutes', 'Total night minutes']

df.groupby(['Churn'])[columns_to_show].agg([len, np.mean, np.std, np.min, np.max]).T

In [ ]:
df.groupby(['Churn'])[columns_to_show].max().T

## Сводные таблицы

### Таблицы сопряженности

Допустим, мы хотим посмотреть, как наблюдения в нашей выборке распределены в контексте двух признаков — Churn и International plan. Для этого мы можем построить таблицу сопряженности, воспользовавшись методом crosstab:

In [ ]:
pd.crosstab(df['Churn'], df['International plan'])

In [ ]:
pd.crosstab(df['Churn'], df['Voice mail plan'], normalize=True)

### Сводные таблицы

In [ ]:
df['Area code'].unique()

 В Pandas за сводные таблицы отвечает метод pivot_table, который принимает в качестве параметров:

    values – список переменных, по которым требуется рассчитать нужные статистики,
    index – список переменных, по которым нужно сгруппировать данные,
    aggfunc — то, что нам, собственно, нужно посчитать по группам — сумму, среднее, максимум, минимум или что-то ещё.


In [ ]:
df.pivot_table(['Total day calls', 'Total eve calls', 'Total night calls'], 
['Area code'], aggfunc='mean')

## Преобразование датафреймов

### Добавление столбцов

In [ ]:
df['Total charge'] = df['Total day charge'] + df['Total eve charge'] + df['Total night charge'] + df['Total intl charge']

df.head(1).T

### Удаление столбцов

In [ ]:
df = df.drop(['Total charge'], axis=1) 

### Удаление строк

In [ ]:
df.drop([1, 2]).head()

## Итоги первичного анализа данных

### Подключение роуминга

In [ ]:
pd.crosstab(df['Churn'], df['International plan'], margins=True)

In [ ]:
pd.crosstab(df['Churn'], df['International plan']).T.plot(kind="bar");

Вывод?

### Звонки в техподдержку

In [ ]:
pd.crosstab(df['Churn'], df['Customer service calls'], margins=True)

In [ ]:
pd.crosstab(df['Churn'], df['Customer service calls']).T.plot(kind="bar");

TODO: Вывод

## Задание

1. Удалить из датасета лишние признаки
2. Сохранить получившийся датасет в csv-файл, используя в качестве разаделителя символ `|`
3. Загрузить данные из только что сохраненного файла
4. Проанализировать влияние признака `State` на отток